In [ ]:
# Lets begin by setting up our toy problem again.
from pyplasmaopt import *
nfp = 2
(coils, currents, magnetic_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=10, at_optimum=False)
stellarator = CoilCollection(coils, currents, nfp, True)

PyPlasmaOpt makes is easy to create pertubed coils. Perturbations are governed by two user parameters:
- a length scale for the perbutation (short = high frequence perturbance, long=low frequence pertubations)
- a magnitude of the perturbation.
Below you can see a snipped of code that creates a pertubed coil and first plots the x, y, and z component of the pertubation and the plots the actual perturbed coil.

In [ ]:
length_scale_perturb = 0.05 # play with parameter 
sigma_perturb = 0.1 # and with this one too
n_derivs = 1 # Set to 1 for faster experimentation, later we need to set this to 3 
             # (since we also need the tangent, curvature and torsion of the perturbed coils).
sampler = GaussianSampler(coils[0].points, length_scale=length_scale_perturb,
                          sigma=sigma_perturb, n_derivs=n_derivs)

process = sampler.sample()[0]
%matplotlib inline
import matplotlib.pyplot as plt
# Let's have a look at the three components of the pertubation.
plt.plot(process)
plt.show()
# And now let's create a perturbed coil.
perturbed_coil = GaussianPerturbedCurve(stellarator.coils[0], sampler)
ax = stellarator.coils[0].plot(show=False)
perturbed_coil.plot(ax=ax, show=True)

## Task 1:
Modify `length_scale_perturb` and `sigma_perturb` and observe the effect on the resulting pertubation.

We will now create a series of perturbed stellarators and create an objective for each of these perturbations.

In [ ]:
iota_target = 0.103
coil_length_target = 4.398229715025710
magnetic_axis_length_target = 6.356206812106860
eta_bar = -2.25
(coils, currents, magnetic_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=10, at_optimum=False)
stellarator = CoilCollection(coils, currents, nfp, True)
det_objective = SimpleNearAxisQuasiSymmetryObjective(
        stellarator, magnetic_axis, iota_target, eta_bar=eta_bar,
        coil_length_target=coil_length_target,
        magnetic_axis_length_target=magnetic_axis_length_target)

sampler = GaussianSampler(coils[0].points, length_scale=0.2, sigma=0.003, n_derivs=3)
nsamples = 25
objectives = []    
for i in range(nsamples):
    (coils, currents, magnetic_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=10, at_optimum=False)
    stellarator = CoilCollection(coils, currents, nfp, True)
    stellarator.coils = [GaussianPerturbedCurve(coil, sampler) for coil in stellarator.coils]
    objectives.append(
        SimpleNearAxisQuasiSymmetryObjective(
            stellarator, magnetic_axis, iota_target, eta_bar=eta_bar,
            coil_length_target=coil_length_target,
            magnetic_axis_length_target=magnetic_axis_length_target))


Before doing any stochastic optimisation, we first run the deterministic optimisation and then evaluate the objective for each of the perturbed stellarators at the obtained minimum.

In [ ]:
from scipy.optimize import minimize
def scipy_fun(x):
    det_objective.update(x)
    res = det_objective.res
    dres = det_objective.dres
    return res, dres

res = minimize(scipy_fun, det_objective.x0, jac=True, method='bfgs', tol=1e-20,
               options={"maxiter": 500},
               callback=det_objective.callback)
vals = []
for obj in objectives:
    obj.update(res.x)
    vals.append(obj.res)
print(vals)

In [ ]:
plt.hist(vals)
plt.show()
print('Mean objective', np.mean(vals))

Despite the deterministic objective being reduced down to $\sim 10^{-8}$, the objective for the perturbed coils varies around $10^{-4}-10^{-3}$. This shows the impact of coil errors on the quasi symmetry of the magnetic field.

Let's also quickly plot the stellarator for later comparison.

In [ ]:
det_objective.update(res.x)
plot_stellarator(det_objective.stellarator, axis=det_objective.ma)

We will now build the stochastic objective. To do this, we compute the objective value and the gradient for each of the objectives corresponding to a perturbed stellarator.

In [ ]:
Jvals_stoch = []

def scipy_fun_stoch(x):
    res = 0.
    dres = np.zeros_like(x)
    for obj in objectives:
        obj.update(x)
        res += obj.res
        dres += obj.dres
    res *= 1./nsamples
    dres *= 1./nsamples
    Jvals_stoch.append(res)
    return res, dres

res_stoch = minimize(scipy_fun_stoch, det_objective.x0, jac=True, method='bfgs', tol=1e-20,
               options={"maxiter": 500, "maxcor": 100},
               callback=objectives[0].callback)
print(res)

In [ ]:
plt.semilogy(Jvals_stoch)
plt.show()

Let's plot the objective values at the found minimum. Comparing this with the histogram above, we can see that the objective is a fair bit smaller on average. 

In [ ]:
vals = []
for obj in objectives:
    obj.update(res_stoch.x)
    vals.append(obj.res)
plt.hist(vals)
plt.show()
print('Mean objective', np.mean(vals))

We approximated the mean of the objective by drawing a fixed number of perturbed coils from the distribution and then minimising for these coils. However, we might ask how good the found minimum is for different realisations of the noise. In other words, did we maybe do a good job for these particular perturbations, but are terrible for others?
To do this, we perform an out-of-sample test. We draw 'fresh' pertubations and compute the mean objective value for them.

In [ ]:
n_outofsample = 100
objectives_outofsample = []    
for i in range(n_outofsample):
    (coils, currents, magnetic_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=10, at_optimum=False)
    stellarator = CoilCollection(coils, currents, nfp, True)
    stellarator.coils = [GaussianPerturbedCurve(coil, sampler) for coil in stellarator.coils]
    objectives_outofsample.append(
        SimpleNearAxisQuasiSymmetryObjective(
            stellarator, magnetic_axis, iota_target, eta_bar=eta_bar,
            coil_length_target=coil_length_target,
            magnetic_axis_length_target=magnetic_axis_length_target))
in_sample_values = []
for obj in objectives:
    obj.update(res_stoch.x)
    in_sample_values.append(obj.res)
mean_insample = np.mean(in_sample_values)

out_of_sample_values = []
for obj in objectives_outofsample:
    obj.update(res_stoch.x)
    out_of_sample_values.append(obj.res)
mean_outofsample = np.mean(out_of_sample_values)
print('In-sample =', mean_insample, 'vs out-of-sample =', mean_outofsample)

**Observation**: We can see that the out-of-sample mean is slightly worse than the in-sample mean, but still significantly better than what we obtained from deterministic optimisation.

### Task:
Change the `nsamples` variable in the third code cell to a low number, say 4, and rerun the stochastic optimisation and the in-sample vs out-of-sample comparison. How does it change?